In [156]:
import pandas as pd
import numpy as np
import functools, collections
from tqdm.notebook import tqdm
tqdm.pandas()
import warnings
warnings.filterwarnings("ignore")

In [40]:
d = {'metro manila': [3,5,7,7],
                   'luzon': [5,5,7,7],
                   'visayas': [7,7,7,7],
                   'mindanao': [7,7,7,7]}
index_row = ['metro manila', 'luzon', 'visayas', 'mindanao']

In [41]:
SLA_matrix = pd.DataFrame(data=d, index=index_row)

In [69]:
df = pd.read_csv('delivery_orders_march.csv')

In [72]:
# Transform buyer and seller address to simplified area
def simplify_add(list_of_add):
    for word in index_row:
        if word in list_of_add.lower():
            return(word)
        
df['buy_add'] = df['buyeraddress'].apply(simplify_add)
df['sell_add'] = df['selleraddress'].apply(simplify_add)

In [152]:
def get_SLA(buy_add, sell_add):
    SLA = SLA_matrix.loc[buy_add][sell_add]
    return SLA

In [153]:
# Get corresponding SLA for each row
df['first_attempt'] = df[['buy_add','sell_add']].progress_apply(lambda x: get_SLA(*x), axis=1)

In [154]:
import time
def change_time(time_list):
    new_time = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time_list))
    return new_time

In [157]:
df['pick'] = df['pick'].apply(change_time)
df['1st_deliver_attempt'] = df['1st_deliver_attempt'].apply(change_time)
df['2nd_deliver_attempt'] = df['2nd_deliver_attempt'].fillna(0)
df['2nd_deliver_attempt'] = df['2nd_deliver_attempt'].apply(change_time)

In [158]:
df['pick'] = pd.to_datetime(df['pick'])
df['1st_deliver_attempt'] = pd.to_datetime(df['1st_deliver_attempt'])
df['2nd_deliver_attempt'] = pd.to_datetime(df['2nd_deliver_attempt'])

In [159]:
df['pick_date_only'] = df['pick'].dt.date
df['1st_deliver_attempt_date'] = df['1st_deliver_attempt'].dt.date
df['2nd_deliver_attempt_date'] = df['2nd_deliver_attempt'].dt.date

In [160]:
df['1st_diff'] = df['1st_deliver_attempt_date'] - df['pick_date_only']
df['1st_diff'] = df['1st_diff'].dt.days

In [161]:
def num_working_days(start,end):
    valid_calendar = np.busdaycalendar(weekmask='1111110', holidays=['2020-03-08','2020-03-25','2020-03-30','2020-03-31'])
    valid_daycount = np.busday_count(start, end, busdaycal=valid_calendar)
    return valid_daycount

In [162]:
df.rename(columns={'1st_deliver_attempt_date': 'first_attempt_date',
                  '2nd_deliver_attempt_date': 'second_attempt_date'}, inplace=True)

In [163]:
# new_sample['actual_day_used'] = new_sample.apply(lambda x: num_working_days(x.pick_date_only, x.first_attempt_date, axis=1))
df['actual_day_used'] = num_working_days(df['pick_date_only'], df['first_attempt_date'])
df['second_actual_day_used'] = num_working_days(df['first_attempt_date'], df['second_attempt_date'])

In [164]:
df['first_is_late'] = df['actual_day_used'] > df['first_attempt']
df['second_is_late'] = df['second_actual_day_used'] > 3
df['is_late'] = (df['second_is_late']) | (df['first_is_late'])

In [165]:
df_output = df[['orderid','is_late']]
df_output

,orderid,is_late
0,2215676524,False
1,2219624609,False
2,2220979489,False
3,2221066352,False
4,2222478803,False
...,...,...
3176308,31504087640510,False
3176309,31504147352227,False
3176310,31504462290482,False
3176311,31504851495943,False


In [166]:
df_output['is_late'] = df_output['is_late'].astype(int)

In [167]:
df_output['is_late'].value_counts()

0    2935865
1     240448
Name: is_late, dtype: int64

In [104]:
df_output.to_csv('submission.csv', index=False, header=True)